In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib import rc
import seaborn as sns
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.metrics import explained_variance_score

plt.rcParams["axes.unicode_minus"] = False
rc("font", family = "Arial Unicode MS")
%matplotlib inline : 실행을 했을 때 그래프가 바로 나타나게 하는 옵션
get_ipython().run_line_magic("matplotlib","inline")

import warnings
warnings.filterwarning("ignore")

UsageError: unrecognized arguments: : 실행을 했을 때 그래프가 바로 나타나게 하는 옵션


In [ ]:
train = pd.read_csv('FIFA_train.csv')
train.head()

In [ ]:
train.info()

FIFA_train.csv / FIFA_test.csv
 id : 선수 고유의 아이디
 
 name : 이름(object)
 
 age : 나이
 
 continent : 선수들의 국적이 포함되어 있는 대륙입니다(object)
 
 contract_until : 선수의 계약기간이 언제까지인지 나타내어 줍니다(object)
 
 position : 선수가 선호하는 포지션입니다. ex) 공격수, 수비수 등(object)
 
 prefer_foot : 선수가 선호하는 발입니다. ex) 오른발(object)
 
 reputation : 선수가 유명한 정도입니다. ex) 높은 수치일 수록 유명한 선수
 
 stat_overall : 선수의 현재 능력치 입니다.
 
 stat_potential : 선수가 경험 및 노력을 통해 발전할 수 있는 정도입니다.
 
 stat_skill_moves : 선수의 개인기 능력치 입니다.
 
 value : FIFA가 선정한 선수의 이적 시장 가격 (단위 : 유로) 입니다
 


In [ ]:
print("이 데이터는 축구선수 {:d}명의 데이터이며 null값은 존재하지 않는다".format(len(train['id'])))
print("object형 변수는 {:d}개이며, 숫자형 변수는 {:d}개이다.".format(len(train.dtypes=="object"), len(train.dtypes=="int64")+len(train.dtypes=="float64")))

In [ ]:
numtype=["int64","float64"]
numCol=[]
for col, x in zip(train.dtypes.index, train.dtypes):
    if x in numtype:
        numCol.append(col)
numCol

In [ ]:
numCol.remove('id')

In [ ]:
def novarplot(var):
    plt.boxplot(train[var], sym="bo")
    plt.title("Boxplot   " + var)
    plt.xlabel(var)
    plt.show()

In [3]:
for x in numCol:
    novarplot(x)

NameError: name 'numCol' is not defined

In [ ]:
for x in numCol:
    plt.hist(train[x])
    plt.show()

In [ ]:
#value 변환
#1) 2018년 환율 기준으로 won으로 바꿈


In [ ]:
train['value_won'] = train['value'].apply(lambda x: x*1299.65).astype(np.int64)
sns.distplot(train['value_won'])

In [ ]:
# # 편향 처리
# https://towardsdatascience.com/top-3-methods-for-handling-skewed-data-1334e0debf45
# https://www.statisticshowto.com/probability-and-statistics/normal-distributions/box-cox-transformation/
# https://dining-developer.tistory.com/18
# 1) 로그 처리
#https://suppppppp.github.io/posts/Why-Series-MDM-1/
# 2) 루트 처리

# 3) Box-Cox 변환

In [ ]:

train['log_value'] = np.log1p(train['value'])
train['root_value'] = np.sqrt(train['value'])
train['boxcox_value'] = pd.Series(stats.boxcox(train['value'])[0])

In [ ]:
# 왜도 (Skewness)
# - 분포의 비대칭도.

# - 정규분포 = 왜도 0

# - 왼쪽으로 치우침 = 왜도 > 0

# - 오른쪽으로 치우침 = 왜도 < 0

In [ ]:
print("로그변환      :   ", train['log_value'].skew())
print("루트변환      :   ", train['root_value'].skew())
print("box-cox변환  :   ", train['boxcox_value'].skew())

In [ ]:
# 그래프 확인
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

sns.distplot(train['log_value'], ax=ax[0])
sns.distplot(train['root_value'], ax=ax[1])
sns.distplot(train['boxcox_value'], ax=ax[2])

In [ ]:
train.info()

In [ ]:
novarplot('log_value')
novarplot('root_value')
novarplot('boxcox_value')

In [ ]:
print("log 변환     >> 평균 : {:.4f} 편차 : {:.4f}  최소값 : {:.4f}  최대값 : {:.4f} ".format(np.mean(train['log_value']), np.std(train['log_value']), np.min(train['log_value']), np.max(train['log_value'])))
print("root 변환    >> 평균 : {:.2f} 편차 : {:.2f}  최소값 : {:.2f}  최대값 : {:.2f} ".format(np.mean(train['root_value']), np.std(train['root_value']), np.min(train['root_value']), np.max(train['root_value'])))
print("box-cox 변환 >> 평균 : {:.4f} 편차 : {:.4f}  최소값 : {:.4f}  최대값 : {:.4f} ".format(np.mean(train['boxcox_value']), np.std(train['boxcox_value']), np.min(train['boxcox_value']), np.max(train['boxcox_value'])))


In [ ]:
train.head()

In [ ]:
# # object Unique
#  continent : 선수들의 국적이 포함되어 있는 대륙입니다(object)
 
#  contract_until : 선수의 계약기간이 언제까지인지 나타내어 줍니다(object)
 
#  position : 선수가 선호하는 포지션입니다. ex) 공격수, 수비수 등(object)
 
#  prefer_foot : 선수가 선호하는 발입니다. ex) 오른발(object)

In [ ]:
objCol = train.dtypes[train.dtypes=="object"].index
objCol = list(objCol)

In [ ]:
objCol.remove('name')

In [ ]:
objCol

In [ ]:
for x in objCol:
    print("변수 : ", x, train[x].unique(), end="\n")
    print(" >>>> 항목 개수 : ", len(train[x].unique()))

In [ ]:
train['contract_until']=train['contract_until'].map(lambda x: x[-4:])

In [ ]:
train['contract_until'].unique()

In [ ]:
numCol.append('log_value')
numCol.append('boxcox_value')

In [ ]:
numCol

In [ ]:
colormap = plt.cm.PuBu
plt.figure(figsize=(10, 8))
plt.title("Person Correlation of Features", y = 1.05, size = 15)
sns.heatmap(train[numCol].astype(float).corr(), linewidths = 0.1, vmax = 1.0,
           square = True, cmap = colormap, linecolor = "white", annot = True, annot_kws = {"size" : 16})


In [ ]:
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    x_data_basic.values, i) for i in range(x_data_basic.shape[1])]
vif["feature"] = x_data_basic.columns
vif

In [ ]:
print("상관계수는 stat_potential(잠재력) & stat_overall(현재 능력치)이 0.67로 가장 상관관계가 크다.")
print("이 두변수는 변환된 value(log/boxcox 기준)와는 stat_overallDL 0.94로 stat_potential 0.79보다 높은 것으로 나타났으나,")
print("VIF를 확인하였을 때, 다중공선성의 가능성은 없는 것으로 확인하였다.", end='\n\n')

print("두번째로 높은 상관계수는 reputation(유명도) & stat_overall(현재능력치)가 0.52로 높게 나타났다.")

In [ ]:
#Object변수와 Num변수 함께 boxplot 그리기

for obj in objCol:
    for num in numCol:
        sns.boxplot(x=obj,y=num,data=train)
        plt.title("boxplot  "+obj+" & "+num)
        plt.xlabel(obj)
        plt.ylabel(num)
        plt.show()


In [ ]:
for a in numCol:
    for b in numCol:
        sns.scatterplot(x=a,y=b,data=train)
        plt.title("scatterplot  "+a+" & "+b)
        plt.xlabel(a)
        plt.ylabel(b)
        plt.show()
            

In [ ]:
sns.pairplot(train)
plt.show()

In [ ]:
train.head()

In [ ]:
v

In [ ]:
y_data = train['value'].copy()
y_data_log = train['log_value'].copy()
y_data_boxcox = train['boxcox_value'].copy()

In [ ]:
x_data.head()

In [ ]:
x_data = pd.get_dummies(x_data)

In [ ]:
x_data.head()

In [ ]:
#100점 만점 변수 5점 변수로 변환
x_data['stat_overall']=x_data['stat_overall']/100*5
x_data['stat_potential']=x_data['stat_potential']/100*5

In [ ]:
x_data.head()

In [ ]:
x_data['age_minmax'] = (x_data['age']-x_data['age'].min())/(x_data['age'].max()-x_data['age'].min())
x_data['age_std'] = (x_data['age']-x_data['age'].mean())/x_data['age'].std()

In [ ]:
print("age_minmax     >> 평균 : {:.4f} 편차 : {:.4f}  최소값 : {:.4f}  최대값 : {:.4f} ".format(np.mean(x_data['age_minmax']), np.std(x_data['age_minmax']), np.min(x_data['age_minmax']), np.max(x_data['age_minmax'])))
print("age_std   >> 평균 : {:.2f} 편차 : {:.2f}  최소값 : {:.2f}  최대값 : {:.2f} ".format(np.mean(x_data['age_std']), np.std(x_data['age_std']), np.min(x_data['age_std']), np.max(x_data['age_std'])))

In [ ]:
x_data_basic = x_data.copy()
x_data_minmax = x_data.copy()
x_data_std = x_data.copy()

In [ ]:
del x_data_basic['age_minmax']
del x_data_basic['age_std']
del x_data_minmax['age']
del x_data_minmax['age_std']
del x_data_std['age']
del x_data_std['age_minmax']

In [ ]:
x_data_std.head()

In [ ]:
- linear regression
1) age를 그냥 넣어서 y_data/y_data_log/y_data_boxcox 
2) age_minmax 넣어서 y_data/y_data_log/y_data_boxcox
3) age_std 넣어서 y_data/y_data_log/y_data_boxcox

In [ ]:
- random forest
1) age를 그냥 넣어서 y_data/y_data_log/y_data_boxcox 
2) age_minmax 넣어서 y_data/y_data_log/y_data_boxcox
3) age_std 넣어서 y_data/y_data_log/y_data_boxcox

In [ ]:
-xgboost
1) age를 그냥 넣어서 y_data/y_data_log/y_data_boxcox 
2) age_minmax 넣어서 y_data/y_data_log/y_data_boxcox
3) age_std 넣어서 y_data/y_data_log/y_data_boxcox

In [ ]:
#linear regression
x_name = []
y_name = []
tr_rmse = []
test_rmse = []
train_score=[]
test_score = []

x_dataset = [x_data_basic, x_data_minmax, x_data_std]
y_dataset = [y_data, y_data_log, y_data_boxcox]

x_list = ["x_data_basic", "x_data_minmax", "x_data_std"]
y_list = ["y_data", "y_data_log", "y_data_boxcox"]

In [ ]:
for i, a in enumerate(x_dataset):
    for j, b in enumerate(y_dataset):
        X_train, X_test, y_train, y_test = train_test_split(a, b, test_size=0.3, random_state=9)

        reg = LinearRegression()
        reg.fit(X_train, y_train)

        pred_tr = reg.predict(X_train)
        pred_test = reg.predict(X_test)
        rmse_tr = (np.sqrt(mean_squared_error(y_train, pred_tr)))
        rmse_test = (np.sqrt(mean_squared_error(y_test, pred_test)))

        x_name.append(x_list[i])
        y_name.append(y_list[j])
        tr_rmse.append(rmse_tr)
        test_rmse.append(rmse_test)
        train_score.append(reg.score(X_train,y_train))
        test_score.append(reg.score(X_test, y_test))
        print(len(x_name), len(y_name), len(tr_rmse), len(test_rmse), len(train_score),len(test_score))

In [ ]:
df_linear = pd.DataFrame({'X_data' : x_name,
                  'y_data' : y_name,
                  'train_rmse' : tr_rmse,
                  'test_rmse' : test_rmse,
                  'train_score' : train_score,
                  'test_score' : test_score})


In [ ]:
display(df_linear)

In [ ]:
x_dataset[1]

In [ ]:
#randomforest
x_name = []
y_name = []
tr_rmse = []
test_rmse = []
train_score=[]
test_score = []

x_dataset = [x_data_basic, x_data_minmax, x_data_std]
y_dataset = [y_data, y_data_log, y_data_boxcox]

x_list = ["x_data_basic", "x_data_minmax", "x_data_std"]
y_list = ["y_data", "y_data_log", "y_data_boxcox"]

In [ ]:
for i, a in enumerate(x_dataset):
    for j, b in enumerate(y_dataset):
        X_train, X_test, y_train, y_test = train_test_split(a, b, test_size=0.3, random_state=9)

        rfref = RandomForestRegressor()
        rfref.fit(X_train, y_train)

        pred_tr = rfref.predict(X_train)
        pred_test = rfref.predict(X_test)
        rmse_tr = (np.sqrt(mean_squared_error(y_train, pred_tr)))
        rmse_test = (np.sqrt(mean_squared_error(y_test, pred_test)))

        x_name.append(x_list[i])
        y_name.append(y_list[j])
        tr_rmse.append(rmse_tr)
        test_rmse.append(rmse_test)
        train_score.append(rfref.score(X_train,y_train))
        test_score.append(rfref.score(X_test, y_test))
        print(len(x_name), len(y_name), len(tr_rmse), len(test_rmse), len(train_score),len(test_score))

In [ ]:
df_randomforest = pd.DataFrame({'X_data' : x_name,
                  'y_data' : y_name,
                  'train_rmse' : tr_rmse,
                  'test_rmse' : test_rmse,
                  'train_score' : train_score,
                  'test_score' : test_score})


In [ ]:
display(df_randomforest)

In [ ]:
#xgboost
x_name = []
y_name = []
tr_rmse = []
test_rmse = []
train_score=[]
test_score = []

x_dataset = [x_data_basic, x_data_minmax, x_data_std]
y_dataset = [y_data, y_data_log, y_data_boxcox]

x_list = ["x_data_basic", "x_data_minmax", "x_data_std"]
y_list = ["y_data", "y_data_log", "y_data_boxcox"]

In [ ]:
for i, a in enumerate(x_dataset):
    for j, b in enumerate(y_dataset):
        
        X_train, X_test, y_train, y_test = train_test_split(a, b, test_size=0.3, random_state=9)
        
        xgb = xgboost.XGBRegressor(n_estimators=100,learning_rate=0.08,gamma=0,subsample=0.75,colsample_bytree=1,max_depth=7)
        xgb.fit(X_train, y_train)

        pred_tr = xgb.predict(X_train)
        pred_test = xgb.predict(X_test)
        rmse_tr = (np.sqrt(mean_squared_error(y_train, pred_tr)))
        rmse_test = (np.sqrt(mean_squared_error(y_test, pred_test)))

        x_name.append(x_list[i])
        y_name.append(y_list[j])
        tr_rmse.append(rmse_tr)
        test_rmse.append(rmse_test)
        train_score.append(xgb.score(X_train,y_train))
        test_score.append(xgb.score(X_test, y_test))
        #print(len(x_name), len(y_name), len(tr_rmse), len(test_rmse), len(train_score),len(test_score))
        print(x_list[i], y_list[j], xgboost.plot_importance(xgb))

In [ ]:
df_xgboost = pd.DataFrame({'X_data' : x_name,
                  'y_data' : y_name,
                  'train_rmse' : tr_rmse,
                  'test_rmse' : test_rmse,
                  'train_score' : train_score,
                  'test_score' : test_score})


In [ ]:
display(df_xgboost)

In [ ]:
df_linear['algorithm']="linear regression"
df_randomforest['algorithm']="randomforest"
df_xgboost['algorithm']="xgboost"

In [ ]:
df_linear

In [ ]:
result = pd.concat([df_linear, df_randomforest, df_xgboost], ignore_index=True)

In [ ]:
result

In [ ]:
result.sort_values(by=['test_score'], ascending=False)

In [ ]:
test = pd.read_csv('FIFA_test.csv')

In [ ]:
test.head()

In [ ]:
test['contract_until'].unique()

In [ ]:
del test['id']
del test['name']

In [ ]:
test['contract_until']=test['contract_until'].map(lambda x: x[-4:])

In [ ]:
x_test = pd.get_dummies(test)

#100점 만점 변수 5점 변수로 변환
x_test['stat_overall']=x_test['stat_overall']/100*5
x_test['stat_potential']=x_test['stat_potential']/100*5

In [ ]:
x_test['age_minmax'] = (x_test['age']-x_data['age'].min())/(x_data['age'].max()-x_data['age'].min())
x_test['age_std'] = (x_test['age']-x_data['age'].mean())/x_data['age'].std()

In [ ]:
x_test_basic = x_test.copy()
x_test_minmax = x_test.copy()
x_test_std = x_test.copy()

In [ ]:
del x_test_basic['age_minmax']
del x_test_basic['age_std']
del x_test_minmax['age']
del x_test_minmax['age_std']
del x_test_std['age']
del x_test_std['age_minmax']

In [ ]:
x_test_set = [x_test_basic, x_test_minmax, x_test_std]

In [ ]:
reg_pred_basic = reg.predict(x_test_basic)
reg_pred_minmax = reg.predict(x_test_minmax)
reg_pred_std = reg.predict(x_test_std)

In [ ]:
x_test_basic.head()

In [ ]:
reg_pred_basic

In [ ]:
x_test_basic.loc[:,'prediction'] = reg_pred_basic
x_test_minmax.loc[:,'prediction'] = reg_pred_minmax
x_test_std.loc[:,'prediction'] = reg_pred_std

In [ ]:
x_test_basic

In [ ]:
x_test_basic.loc[:,'id'] = test['id']
x_test_minmax.loc[:,'id'] = test['id']
x_test_std.loc[:,'id'] = test['id']

In [ ]:
x_test_minmax.head()

In [ ]:
#원본값 반환
origin = np.log(x)
log_plus1 = np.log(x + 1)
log_1p = np.log1p(x)

print("origin -> np.exp(origin)")
print("origin -> np.exp(log_plus1)")
print("origin -> np.exp(log_1p)")